# Lab 6

You are tasked with evaluating card counting strategies for black jack. In order to do so, you will use object oriented programming to create a playable casino style black jack game where a computer dealer plays against $n$ computer players and possibily one human player. If you don't know the rules of blackjack or card counting, please google it. 

A few requirements:
* The game should utilize multiple 52-card decks. Typically the game is played with 6 decks.
* Players should have chips.
* Dealer's actions are predefined by rules of the game (typically hit on 16). 
* The players should be aware of all shown cards so that they can count cards.
* Each player could have a different strategy.
* The system should allow you to play large numbers of games, study the outcomes, and compare average winnings per hand rate for different strategies.

1. Begin by creating a classes to represent cards and decks. The deck should support more than one 52-card set. The deck should allow you to shuffle and draw cards. Include a "plastic" card, placed randomly in the deck. Later, when the plastic card is dealt, shuffle the cards before the next deal.

In [ ]:
import random

class Card: #individual playing cards  with their suits and values 
    SUITS = ['Spades', 'Hearts', 'Diamonds', 'Clubs']
    FACES = ['2', '3', '4', '5', '6', '7', '8', '9', '10', 'J', 'Q', 'K', 'A']

    def __init__(self, suit, face):
        self.suit = suit
        self.face = face

    def __repr__(self): #will output the representation of the card ex: "6 of Diamonds"
        return f"{self.face} of {self.suit}"

class Deck: #a deck of cards to contain all of the cards
    def __init__(self, num_decks=1):
        self.cards = [Card(suit, face) for suit in Card.SUITS for face in Card.FACES] * num_decks
        self.plastic_card_index = random.randint(0, len(self.cards) - 1) # a way for the plastic card to be identify
        random.shuffle(self.cards) #shuffles the cards 

    def draw(self): #drawing a card from the deck
        card = self.cards.pop(self.plastic_card_index) 
        self.plastic_card_index = random.randint(0, len(self.cards) - 1) # a way to show that a card has been removed from the deck
        random.shuffle(self.cards) #makes sure that the cards shuffled after a card is drawn
        return card

    def shuffle(self): # shuffling the cards
        random.shuffle(self.cards)

    def __len__(self):
        return len(self.cards)

2. Now design your game on a UML diagram. You may want to create classes to represent, players, a hand, and/or the game. As you work through the lab, update your UML diagram. At the end of the lab, submit your diagram (as pdf file) along with your notebook. 

3. Begin with implementing the skeleton (ie define data members and methods/functions, but do not code the logic) of the classes in your UML diagram.

 1. Card:
    Attributes:
       -suit: The suit of the card (e.g., "Spades", "Hearts", etc.)
       -face: The face value of the card (e.g., "2", "3", "Ace", etc.)
    Methods:
        -value(): Returns the numerical value of the card (e.g., 2, 3, 10, etc.)
 2. Hand:
    Attributes:
        -cards: A list to hold the cards in the hand
    Methods:
        -add_card(card): Adds a card to the hand
        -value(): Calculates and returns the total value of the hand
        -bust(): Checks if the hand has busted (i.e., total value > 21)
        -is_blackjack(): Checks if the hand has a blackjack (i.e., 21 with 2 cards)
 3. Player:
    Attributes:
        -name: The name of the player
        -hand: An instance of the Hand class to represent the player's hand
        -chips: The number of chips/money the player has
    Methods:
        -hit(deck): Draws a card from the deck and adds it to the player's hand
        -stand(): Indicates that the player will not draw any more cards
        -win(amount): Updates the player's chips/money when they win a bet
        -lose(amount): Updates the player's chips/money when they lose a bet
        -push(): Handles a tie (no win or loss) in a game
 4. Dealer:
        -Inherits from Player
    Methods:
        -hit(deck): Determines if the dealer should hit based on their hand value
        -stand(): Determines if the dealer should stand based on their hand value
 5. HumanPlayer:
        -Inherits from Player
    Methods:
        -hit_or_stand(): Allows the human player to choose whether to hit or stand during their turn
 6. Deck:
    Attributes:
        -cards: A list to hold the cards in the deck
        -plastic_card_index: The index of a plastic card in the deck (used for reshuffling)
    Methods:
        -draw(): Draws a card from the deck
        -shuffle(): Shuffles the deck
        -reshuffle(): Reshuffles the deck when the plastic card is drawn
        -remaining_cards(): Returns the number of cards remaining in the deck
7. Game:
    Attributes:
       -players: A list of players participating in the game
       -deck: An instance of the Deck class representing the game deck
    Method:
       -deal_initial_hands(): Deals initial hands to all players
       -play_round(): Executes a round of the game, allowing players to hit or stand
       -determine_winner(): Determines the winner(s) of the game round
       -update_player_chips(): Updates player chips based on the game outcome
       -play(): Controls the flow of the game, including multiple rounds until game end 

4. Complete the implementation by coding the logic of all functions. For now, just implement the dealer player and human player.

In [ ]:
import random

class Card:
    SUITS = ['Spades', 'Hearts', 'Diamonds', 'Clubs']
    FACES = ['2', '3', '4', '5', '6', '7', '8', '9', '10', 'J', 'Q', 'K', 'A']
    def __init__(self, suit, face):
        self.suit = suit
        self.face = face
    def value(self): #setting the values for the faces of the cards
        if self.face in ('J', 'Q', 'K'):
            return 10
        elif self.face == 'A':
            return 11
        else:
            return int(self.face)
    def __repr__(self): # give the representation of the card (ex: 2 of Clubs)
        return f"{self.face} of {self.suit}"

class Hand:
    def __init__(self):
        self.cards = []
    def add_card(self, card):
        self.cards.append(card)
    def value(self):
        total = 0
        aces = 0
        for card in self.cards:
            total += card.value()
            if card.face == 'A':
                aces += 1
        while total > 21 and aces > 0:
            total -= 10
            aces -= 1
        return total
    def bust(self): # seeing if the player has more than 21 cards
        return self.value() > 21
    def is_blackjack(self): #seeing if the player has 21 cards
        return len(self.cards) == 2 and self.value() == 21

class Player:
    def __init__(self, name):
        self.name = name
        self.hand = Hand()
        self.chips = 0
    def hit(self, deck):
        self.hand.add_card(deck.draw())
    def stand(self):
        pass
    def win(self, amount):
        self.chips += amount
    def lose(self, amount):
        self.chips -= amount
    def push(self):
        pass

class DealerPlayer(Player): # methods that the dealer can make while a game is being played
    def __init__(self):
        super().__init__("Dealer")
    def hit(self, deck):
        if self.hand.value() < 17:
            return True
        else:
            return False
    def stand(self):
        if self.hand.value() >= 17:
            return True
        else:
            return False

class HumanPlayer(Player): # provides a method for players to make decisions while playing game
    def __init__(self, name):
        super().__init__(name)
    def hit_or_stand(self):
        while True:
            decision = input("Do you want to hit or stand? (h/s): ").strip().lower()
            if decision in ['h', 'hit']:
                return True
            elif decision in ['s', 'stand']:
                return False
            else:
                print("Invalid input. Please enter 'h' for hit or 's' for stand.")

class Deck:
    def __init__(self, num_decks=1):
        self.cards = [Card(suit, face) for suit in Card.SUITS for face in Card.FACES] * num_decks
        self.plastic_card_index = random.randint(0, len(self.cards) - 1)
        random.shuffle(self.cards)
    def draw(self):
        card = self.cards.pop(self.plastic_card_index)
        self.plastic_card_index = random.randint(0, len(self.cards) - 1)
        random.shuffle(self.cards)
        return card
    def shuffle(self):
        random.shuffle(self.cards)
    def __len__(self):
        return len(self.cards)

class Game:
    def __init__(self, players):
        self.players = players
        self.deck = Deck()
    def deal_initial_hands(self):
        for _ in range(2):
            for player in self.players:
                player.hand.add_card(self.deck.draw())
    def play_round(self): # a loop to see if the player meets the conditions during their turn (bust, blackjack, or neither)
        for player in self.players:
            while True:
                if player == dealer:
                    if player.stand():
                        break
                    elif player.hit(self.deck):
                        player.hit(self.deck)
                else:
                    if player.hand.bust():
                        print(f"{player.name} busts with a score of {player.hand.value()}!")
                        break
                    elif player.hand.is_blackjack():
                        print(f"{player.name} gets a blackjack with a score of {player.hand.value()}!")
                        break
                    else:
                        if player.hit_or_stand():
                            player.hit(self.deck)
                        else:
                            break
# Create players
strategy_player = HumanPlayer("Strategy Player")
dealer = DealerPlayer()
other_players = [Player(f"Player {i+1}") for i in range(3)]
# Create a game with the players
game = Game([strategy_player] + other_players + [dealer])
# Set initial chips for all players
initial_chips = 100
for player in game.players:
    player.chips = initial_chips
# Play 50 rounds or until the strategy player is out of money
rounds_to_play = 50
for _ in range(rounds_to_play):
    # Check if the strategy player is out of money
    if


5.  Test. Demonstrate game play. For example, create a game of several dealer players and show that the game is functional through several rounds.

In [ ]:
import random

# Define the classes (Card, Hand, Player, Deck, Game) here...

# Create players
player1 = Player("Player 1")
player2 = Player("Player 2")
dealer = Player("Dealer")

# Create a game with the players
game = Game([player1, player2, dealer])

# Play several rounds
num_rounds = 3
for round_num in range(num_rounds):
    print(f"Round {round_num + 1}:")
    
    # Deal initial hands
    game.deal_initial_hands()
    
    # Print initial hands
    print("Initial hands:")
    for player in game.players:
        print(f"{player.name}: {player.hand.cards}")
    
    # Play the round
    game.play_round()
    
    # Print the results
    print("Results:")
    for player in game.players:
        if player != dealer:
            print(f"{player.name}'s score: {player.hand.value()}")
    print(f"{dealer.name}'s score: {dealer.hand.value()}")
    print("--------------------------")



6. Implement a new player with the following strategy:

    * Assign each card a value: 
        * Cards 2 to 6 are +1 
        * Cards 7 to 9 are 0 
        * Cards 10 through Ace are -1
    * Compute the sum of the values for all cards seen so far.
    * Hit if sum is very negative, stay if sum is very positive. Select a threshold for hit/stay, e.g. 0 or -2.  

In [ ]:
class CardCounterPlayer(Player):
    def __init__(self, name):
        super().__init__(name)
        self.card_values = {'2': 1, '3': 1, '4': 1, '5': 1, '6': 1,
                            '7': 0, '8': 0, '9': 0,
                            '10': -1, 'J': -1, 'Q': -1, 'K': -1, 'A': -1}
        self.sum_card_values = 0

    def update_sum_card_values(self, card):
        self.sum_card_values += self.card_values[card.face]

    def hit(self, deck):
        super().hit(deck)
        new_card = self.hand.cards[-1]
        self.update_sum_card_values(new_card)
        if self.sum_card_values <= -2:  # Threshold for hitting
            return True
        else:
            return False

    def stand(self):
        if self.sum_card_values >= 0:  # Threshold for standing
            return True
        else:
            return False


7. Create a test scenario where one player, using the above strategy, is playing with a dealer and 3 other players that follow the dealer's strategy. Each player starts with same number of chips. Play 50 rounds (or until the strategy player is out of money). Compute the strategy player's winnings. You may remove unnecessary printouts from your code (perhaps implement a verbose/quiet mode) to reduce the output.

In [ ]:
class DealerPlayer(Player):
    def __init__(self):
        super().__init__("Dealer")

    def hit(self, deck):
        if self.hand.value() < 17:
            return True
        else:
            return False

    def stand(self):
        if self.hand.value() >= 17:
            return True
        else:
            return False

# Create players
strategy_player = CardCounterPlayer("Strategy Player")
dealer = DealerPlayer()
other_players = [Player(f"Player {i+1}") for i in range(3)]

# Create a game with the players
game = Game([strategy_player] + other_players + [dealer])

# Set initial chips for all players
initial_chips = 100
for player in game.players:
    player.chips = initial_chips

# Play 50 rounds or until the strategy player is out of money
rounds_to_play = 50
for _ in range(rounds_to_play):
    # Check if the strategy player is out of money
    if strategy_player.chips <= 0:
        break
    
    # Deal initial hands
    game.deal_initial_hands()
    
    # Play the round
    game.play_round()
    
    # Determine winners and losers
    for player in game.players:
        if player != dealer and player != strategy_player:
            if player.hand.value() > dealer.hand.value() or dealer.hand.bust():
                player.win(10)  # Assuming bet amount of 10 chips
                dealer.lose(10)
            elif player.hand.value() < dealer.hand.value():
                player.lose(10)
                dealer.win(10)
            else:
                player.push()

# Compute strategy player's winnings
strategy_player_winnings = strategy_player.chips - initial_chips
print(f"Strategy player's winnings after {rounds_to_play} rounds: {strategy_player_winnings} chips")



8. Create a loop that runs 100 games of 50 rounds, as setup in previous question, and store the strategy player's chips at the end of the game (aka "winnings") in a list. Histogram the winnings. What is the average winnings per round? What is the standard deviation. What is the probabilty of net winning or lossing after 50 rounds?


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Define a function to play a single game of 50 rounds and return strategy player's chips at the end
def play_single_game():
    # Create players
    strategy_player = HumanPlayer("Strategy Player")
    dealer = DealerPlayer()
    other_players = [Player(f"Player {i+1}") for i in range(3)]
    # Create a game with the players
    game = Game([strategy_player] + other_players + [dealer])
    # Set initial chips for all players
    initial_chips = 100
    for player in game.players:
        player.chips = initial_chips
    # Play 50 rounds
    for _ in range(50):
        # Check if the strategy player is out of money
        if strategy_player.chips <= 0:
            break
        # Deal initial hands
        game.deal_initial_hands()
        # Play the round
        game.play_round()
    # Return strategy player's chips at the end of the game
    return strategy_player.chips
# Play 100 games of 50 rounds each and store strategy player's winnings
winnings = [play_single_game() for _ in range(100)]
# Plot histogram of winnings
plt.hist(winnings, bins=20, color='skyblue', edgecolor='black')
plt.title('Histogram of Strategy Player\'s Winnings')
plt.xlabel('Winnings')
plt.ylabel('Frequency')
plt.show()
# Calculate average winnings per round
average_winnings_per_round = np.mean(winnings) / 50
# Calculate standard deviation of winnings
std_deviation = np.std(winnings)
# Calculate probability of net winning or losing after 50 rounds
prob_net_win = sum([1 for win in winnings if win > initial_chips]) / len(winnings)
prob_net_loss = sum([1 for win in winnings if win < initial_chips]) / len(winnings)
print(f"Average winnings per round: {average_winnings_per_round}")
print(f"Standard deviation of winnings: {std_deviation}")
print(f"Probability of net winning after 50 rounds: {prob_net_win}")
print(f"Probability of net losing after 50 rounds: {prob_net_loss}")


9. Repeat previous questions scanning the value of the threshold. Try at least 5 different threshold values. Can you find an optimal value?

In [ ]:
# Define threshold values to try
threshold_values = [-2, -1, 0, 1, 2]
# Store results for each threshold
results = []
# Play games for each threshold value
for threshold in threshold_values:
    # Play 100 games of 50 rounds each and store strategy player's winnings
    winnings = [play_single_game(threshold) for _ in range(100)]
    # Store results
    results.append({
        'threshold': threshold,
        'winnings': winnings,
        'average_winnings_per_round': np.mean(winnings) / 50,
        'std_deviation': np.std(winnings),
        'prob_net_win': sum([1 for win in winnings if win > initial_chips]) / len(winnings),
        'prob_net_loss': sum([1 for win in winnings if win < initial_chips]) / len(winnings)
    })
# Print results for each threshold value
for result in results:
    print(f"Threshold: {result['threshold']}")
    print(f"Average winnings per round: {result['average_winnings_per_round']}")
    print(f"Standard deviation of winnings: {result['std_deviation']}")
    print(f"Probability of net winning after 50 rounds: {result['prob_net_win']}")
    print(f"Probability of net losing after 50 rounds: {result['prob_net_loss']}")
    print()
# Plot histogram of winnings for each threshold value
plt.figure(figsize=(10, 6))
for result in results:
    plt.hist(result['winnings'], bins=20, alpha=0.5, label=f"Threshold: {result['threshold']}")
plt.title('Histogram of Strategy Player\'s Winnings for Different Threshold Values')
plt.xlabel('Winnings')
plt.ylabel('Frequency')
plt.legend()
plt.show()


10. Create a new strategy based on web searches or your own ideas. Demonstrate that the new strategy will result in increased or decreased winnings. 

In [ ]:
class CardCountingPlayer(Player):
    def __init__(self, name):
        super().__init__(name)
        self.count = 0
    def hit_or_stand(self):
        # If count is positive, more high cards remaining, so player should hit
        if self.count > 0:
            return True
        # If count is negative or zero, more low cards remaining, so player should stand
        else:
            return False
    def update_count(self, card):
        # Update count based on the value of the drawn card
        if card.value in [10, 'J', 'Q', 'K', 'A']:
            self.count -= 1
        elif card.value in [2, 3, 4, 5, 6]:
            self.count += 1
# Create a game with the new strategy player
game = Game([CardCountingPlayer("Card Counting Player")] + [Player(f"Player {i+1}") for i in range(3)] + [DealerPlayer()])
# Play 100 games of 50 rounds each
winnings = [game.play() for _ in range(100)]
# Calculate average winnings per round
average_winnings_per_round = np.mean(winnings) / 50
# Calculate standard deviation of winnings
std_deviation = np.std(winnings)
# Calculate probability of net winning or losing after 50 rounds
prob_net_win = sum([1 for win in winnings if win > 0]) / len(winnings)
prob_net_loss = sum([1 for win in winnings if win < 0]) / len(winnings)
print(f"Average winnings per round: {average_winnings_per_round}")
print(f"Standard deviation of winnings: {std_deviation}")
print(f"Probability of net winning after 50 rounds: {prob_net_win}")
print(f"Probability of net losing after 50 rounds: {prob_net_loss}")
